In [13]:
import pandas as pd
import numpy as np
import os
import sys
import json
import string

from sqlalchemy import create_engine
import psycopg2

In [2]:
master_folder = '/Users/kurniawankesumaputra/Desktop/work/dkatalis/technical_test/'

In [3]:
filename_list = [
    'CRM Events.csv',
    'CRM Call Center Logs.csv',
]

In [43]:
with open(os.path.join(master_folder,'database.json'),'r') as f:
    db_credential = json.load(f)

In [52]:
date_columns = {
    'CRM Events.csv':[
        'date_received'
    ],
    'CRM Call Center Logs.csv':[
        'date_received',
        "serstart",
        "serexit",
        "sertime"
    ]
}

In [79]:
date_columns = [
    "date_received",
    "date_sent_to_company"
]
time_columns = [
    "serstart",
    "serexit",
    "sertime"
]

In [80]:
conn_string = f"postgresql://{db_credential['POSTGRES_USER']}:{db_credential['POSTGRES_PASSWORD']}@{db_credential['POSTGRESS_HOST']}/{db_credential['POSTGRES_DB']}"
db = create_engine(conn_string)
conn = db.connect()
for file in filename_list:
    file_path = os.path.join(master_folder,*['data',file])
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.translate(str.maketrans('', '', string.punctuation)).str.replace(' ','_').str.lower().str.strip()
    table_name = file.split('.csv')[0].lower().strip().translate(str.maketrans('', '', string.punctuation)).replace(' ','_')
    for col in df.columns:
        if col in date_columns:
            df[col] = pd.to_datetime(df[col],errors='coerce')
        elif col in time_columns:
            df[col] = pd.to_datetime(df[col],format="%H:%M:%S",errors='coerce').dt.time
    df.to_sql(table_name, con=conn,schema=db_credential['POSTGRES_SCHEMA'], if_exists='replace', index=False)
    print("-"*50)
conn.close()

--------------------------------------------------
--------------------------------------------------


In [81]:
df.dtypes

date_received    datetime64[ns]
complaint_id             object
rand_client              object
phonefinal               object
vruline                  object
callid                  float64
priority                float64
type                     object
outcome                  object
server                   object
serstart                 object
serexit                  object
sertime                  object
dtype: object

In [ ]:
for file in filename_list:
    file_path = os.path.join(master_folder,*['data',file])
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.translate(str.maketrans('', '', string.punctuation)).str.replace(' ','_').str.lower().str.strip()
    table_name = file.split('.csv')[0].lower().strip().translate(str.maketrans('', '', string.punctuation)).replace(' ','_')
    fields = []
    avro_data = {
        "namespace": f"{table_name}.avro",
        "type": "record",
        "name": table_name,
    }
    for col in df.columns:
        if col in date_columns:
            type_ = "int"
            df[col] = pd.to_datetime(df[col],errors='coerce')
        else:
            type_ = "string"
        
        dat_ = {
            "name": col,
            "type": [
                type_,"null"
            ]
        }
        fields.append(dat_)
    avro_data["field"] = fields
    with open(os.path.join(master_folder,*["avro_schema",f"{table_name}.json"]),'w') as f:
        json.dump(avro_data,f,indent=4)
        f.close()

In [ ]:
crm_event = pd.read_csv